## Лабораторная № 1. Задание 2

In [13]:
import pandas as pd
from urllib.parse import quote

Загрузка html

In [14]:
url = "https://pstu.ru/files/file/Abitur/2025%20final/ФПММ_Б_Н_000007052.html"
encoded_url = quote(url, safe=':/%')
initial_df = pd.read_html(encoded_url)[0]


Функция для чистки таблицы

In [15]:
def clear_table(df):
    col = df.iloc[12].tolist()
    df.columns = col
    df_cleared = df.loc[13:]
    df_cleared = df_cleared.loc[13:]
    df_cleared = df_cleared.reset_index(drop=True)
    df_cleared = df_cleared.dropna(axis=1)
    df_cleared = df_cleared.drop(columns=['№', 'Уникальный код', 'Представление приказа', 'Учебная группа'])
    return df_cleared

Функция для получения предметов

In [16]:
def get_subjects(df):
    return df.columns[1:4].tolist()

Функция для приведения типов

In [17]:
def table_to_numeric(df):
    return df[df.columns].astype(int)

Функции для поиска среднего, минимального и максимального суммарных баллов

In [18]:
def find_average_points(df):
    return (df['Сумма баллов'].mean()).astype(int)

def find_min_points(df):
    return df['Сумма баллов'].min()

def find_max_points(df):
    return df['Сумма баллов'].max()

- Функция для поиска средних баллов по предметам
- Функция для поиска предмета с наивысшим средним

In [19]:
def get_subject_avg_points(df):
    subjects = get_subjects(df)
    avg_points = df[subjects].mean().astype(int)
    return avg_points

Функция для определения количества студентов типа 1 и типа 2

In [20]:
def find_type1_students(df, subjects, subject_avg_points):
    return df[(df[subjects[0]] > subject_avg_points[subjects[0]]) & (df[subjects[1]] > subject_avg_points[subjects[1]]) & (df[subjects[2]] < subject_avg_points[subjects[2]])].shape[0]

def find_type2_students(df, subjects, subject_avg_points):
    return df[(df[subjects[0]] < subject_avg_points[subjects[0]]) & (df[subjects[1]] < subject_avg_points[subjects[1]]) & (df[subjects[2]] > subject_avg_points[subjects[2]])].shape[0]

Функция для построения итоговой таблицы

In [21]:
def create_final_table(initial_df, numeric_df):
    cols = ["Подразделение", "Уровень подготовки", "Направление подготовки/специальность", "Год", "Количество мест", "Предметы ЕГЭ",
            "Средняя сумма баллов", "Min сумма баллов", "Max сумма баллов", "Предмет с высшим средним", "Кол-во студентов тип_1", "Кол-во студентов тип_2"]
    final_df = pd.DataFrame(columns=cols)

    department = initial_df.iloc[4, 0]
    department = department.split(' - ')[1]
    final_df['Подразделение'] = [department]

    level = initial_df.iloc[5, 0]
    level = level.split(' - ')[1]
    final_df['Уровень подготовки'] = [level]

    specialization = initial_df.iloc[6, 0]
    specialization = specialization.split(' - ')[1]
    final_df['Направление подготовки/специальность'] = [specialization]

    year = initial_df.iloc[2, 0]
    year = year[26:30]
    final_df['Год'] = [year]

    place_count = initial_df.iloc[10, 0]
    place_count = place_count[12:15]
    final_df['Количество мест'] = [place_count]

    subjects = get_subjects(numeric_df)
    final_df['Предметы ЕГЭ'] = [subjects]

    final_df['Средняя сумма баллов'] = [find_average_points(numeric_df)]

    final_df['Min сумма баллов'] = [find_min_points(numeric_df)]

    final_df['Max сумма баллов'] = [find_max_points(numeric_df)]

    subject_avg_points = get_subject_avg_points(numeric_df)
    final_df['Предмет с высшим средним'] = [subject_avg_points.idxmax()]

    final_df['Кол-во студентов тип_1'] = [find_type1_students(numeric_df, subjects, subject_avg_points)]

    final_df['Кол-во студентов тип_2'] = [find_type2_students(numeric_df, subjects, subject_avg_points)]

    return final_df

In [22]:
# Вызов функций
cleared_df = clear_table(initial_df)
numeric_df = table_to_numeric(cleared_df)
final_result = create_final_table(initial_df, numeric_df)
display(final_result)

,Подразделение,Уровень подготовки,Направление подготовки/специальность,Год,Количество мест,Предметы ЕГЭ,Средняя сумма баллов,Min сумма баллов,Max сумма баллов,Предмет с высшим средним,Кол-во студентов тип_1,Кол-во студентов тип_2
0,Факультет прикладной математики и механики,Бакалавриат,Информационные системы и технологии,2025,62.,"[АиНМА / Математика, ИТ / Информатика и ИКТ / ...",216,200,248,АиНМА / Математика,4,6
